<a href="https://colab.research.google.com/github/jari-el13/dsc-capstone-project-v2-onl01-dtsc-ft-041320/blob/master/CTGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install ctgan



     |████████████████████████████████| 10.4MB 3.3MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.
  Found existing installation: pandas 1.0.5
    Uninstalling pandas-1.0.5:
      Successfully uninstalled pandas-1.0.5


In [4]:

import ctgan

In [8]:
from ctgan import load_demo

In [42]:
from ctgan import CTGANSynthesizer

In [1]:
import pandas as pd
import numpy as np

In [ ]:
data = load_demo()

In [ ]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [ ]:
data.isna().sum()

age               0
workclass         0
fnlwgt            0
education         0
education-num     0
marital-status    0
occupation        0
relationship      0
race              0
sex               0
capital-gain      0
capital-loss      0
hours-per-week    0
native-country    0
income            0
dtype: int64

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
age               32561 non-null int64
workclass         32561 non-null object
fnlwgt            32561 non-null int64
education         32561 non-null object
education-num     32561 non-null int64
marital-status    32561 non-null object
occupation        32561 non-null object
relationship      32561 non-null object
race              32561 non-null object
sex               32561 non-null object
capital-gain      32561 non-null int64
capital-loss      32561 non-null int64
hours-per-week    32561 non-null int64
native-country    32561 non-null object
income            32561 non-null object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [ ]:
discrete_col = [
                "workclass",
                "education",
                "marital-status",
                "occupation",
                "relationship",
                "race",
                "sex",
                "native-country",
                "income"
]

In [ ]:
ctgan.fit(data, discrete_col, epochs=5)

Epoch 1, Loss G: 1.3493, Loss D: -0.1177
Epoch 2, Loss G: 1.0194, Loss D: -0.0373
Epoch 3, Loss G: 0.3942, Loss D: 0.0197
Epoch 4, Loss G: -0.0634, Loss D: 0.1379
Epoch 5, Loss G: -0.5236, Loss D: -0.0076


In [ ]:
samp = ctgan.sample(100
                    )

In [ ]:
samp.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,53.413,Self-emp-inc,149927,HS-grad,13.0321,Never-married,Adm-clerical,Unmarried,White,Female,-106.668,-5.40855,43.9395,United-States,<=50K
1,54.8632,Private,87060.1,HS-grad,12.8938,Married-civ-spouse,Exec-managerial,Not-in-family,Black,Female,43.5805,-5.52452,18.7398,United-States,<=50K
2,43.9026,Private,77902.6,Bachelors,12.8181,Married-civ-spouse,Prof-specialty,Not-in-family,White,Female,124.352,-1.71944,39.9583,United-States,<=50K
3,35.7426,Private,146842,HS-grad,8.98704,Married-civ-spouse,Adm-clerical,Unmarried,White,Male,18.6858,-3.06091,39.805,United-States,<=50K
4,53.1584,Local-gov,136831,HS-grad,13.8752,Married-civ-spouse,Craft-repair,Husband,White,Female,88.937,-4.20529,22.406,United-States,>50K


In [ ]:
samp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 15 columns):
age               100 non-null object
workclass         100 non-null object
fnlwgt            100 non-null object
education         100 non-null object
education-num     100 non-null object
marital-status    100 non-null object
occupation        100 non-null object
relationship      100 non-null object
race              100 non-null object
sex               100 non-null object
capital-gain      100 non-null object
capital-loss      100 non-null object
hours-per-week    100 non-null object
native-country    100 non-null object
income            100 non-null object
dtypes: object(15)
memory usage: 11.8+ KB


In [2]:
train_data = pd.read_csv('Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_values.csv')
test_data = pd.read_csv('Pump_it_Up_Data_Mining_the_Water_Table_-_Test_set_values.csv')
train_labels = pd.read_csv('Pump_it_Up_Data_Mining_the_Water_Table_-_Training_set_labels.csv')

train = train_data.merge(train_labels, on='id', how='inner')
test_data_copy = test_data.copy()

In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
date_recorded            59400 non-null object
funder                   55765 non-null object
gps_height               59400 non-null int64
installer                55745 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
subvillage               59029 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           56066 non-null object
r

In [7]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [8]:

train_labels['status_group'].value_counts(normalize = True)


functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [9]:
train['installer'].value_counts()
# Five main installers are DWE, Government, RWE, Commu, DANIDA

DWE                        17402
Government                  1825
RWE                         1206
Commu                       1060
DANIDA                      1050
                           ...  
MACK DONALD CONTRSCTOR         1
COWI                           1
DWE/Ubalozi wa Marekani        1
Juma Makulilo                  1
Barry A. Murphy                1
Name: installer, Length: 2145, dtype: int64

In [10]:
# installer put into different categories
def clean_installer(train):
     if train['installer']=='DWE':
         return 'dwe'
     elif train['installer']=='Government':
         return 'government'
     elif train['installer']=='RWE':
         return 'rwe'
     elif train['installer']=='Commu':
         return 'commu'
     elif train['installer']=='DANIDA':
         return 'danida'    
     else:
         return 'other'
    
train['installer']= train.apply(lambda row: clean_installer(row), axis=1)
test_data_copy['installer']= test_data_copy.apply(lambda row: clean_installer(row), axis=1)

In [11]:
train['installer'].value_counts()

other         36857
dwe           17402
government     1825
rwe            1206
commu          1060
danida         1050
Name: installer, dtype: int64

In [13]:
train['funder'].value_counts()

Government Of Tanzania         9084
Danida                         3114
Hesawa                         2202
Rwssp                          1374
World Bank                     1349
                               ... 
Gdp                               1
Asgerali N Bharwan                1
Doner And Com                     1
Qwekwin                           1
Afya Department Lindi Rural       1
Name: funder, Length: 1897, dtype: int64

In [14]:
def funder_cleaning(train):
  if train['funder']=='Government Of Tanzania':
    return 'government'
  elif train['funder']=='Danida':
    return 'danida'
  elif train['funder']=='Hesawa':
    return 'hesawa'
  elif train['funder']=='Rwssp':
    return 'rwssp'
  elif train['funder']=='World Bank':
    return 'world_bank'    
  else:
    return 'other'
    
train['funder']= train.apply(lambda row: funder_cleaning(row), axis=1)
test_data_copy['funder']= test_data_copy.apply(lambda row: funder_cleaning(row), axis=1)

In [15]:
train['funder'].value_counts()

other         42277
government     9084
danida         3114
hesawa         2202
rwssp          1374
world_bank     1349
Name: funder, dtype: int64

In [16]:
train['scheme_management'].value_counts()


VWC                 36793
WUG                  5206
Water authority      3153
WUA                  2883
Water Board          2748
Parastatal           1680
Private operator     1063
Company              1061
Other                 766
SWC                    97
Trust                  72
None                    1
Name: scheme_management, dtype: int64

In [20]:

def scheme_wrangler(row):
     '''Keep top 5 values and set the rest to 'other'. '''
     if row['scheme_management']=='VWC':
         return 'vwc'
     elif row['scheme_management']=='WUG':
         return 'wug'
     elif row['scheme_management']=='Water authority':
         return 'wtr_auth'
     elif row['scheme_management']=='WUA':
         return 'wua'
     elif row['scheme_management']=='Water Board':
         return 'wtr_brd'
     else:
         return 'other'

train['scheme_management'] = train.apply(lambda row: scheme_wrangler(row), axis=1)
test_data_copy['scheme_management'] = test_data_copy.apply(lambda row: scheme_wrangler(row), axis=1)

In [38]:
train.isna().sum()

id                         0
amount_tsh                 0
funder                     0
gps_height                 0
installer                  0
longitude                  0
latitude                   0
wpt_name                   0
num_private                0
basin                      0
subvillage               371
region                     0
region_code                0
district_code              0
lga                        0
ward                       0
population                 0
public_meeting             0
scheme_management          0
permit                     0
construction_year          0
extraction_type_class      0
management                 0
management_group           0
payment_type               0
water_quality              0
quantity                   0
quantity_group             0
source_type                0
source_class               0
waterpoint_type_group      0
status_group               0
dtype: int64

In [22]:
# Will also drop scheme_name feature because it is missing 47.4 of its values in the test dataset
train.drop(['scheme_name'], axis=1, inplace=True)
test_data_copy.drop(['scheme_name'], axis=1, inplace=True)

In [23]:
# Since public_meeting gives us true of false to rather the well if for public meeting and the
# majority of the vales are true we will fill the 5.9 percent of the missing values with true.
train['public_meeting'] = train['public_meeting'].fillna('True')
test_data_copy['public_meeting'] = test_data_copy['public_meeting'].fillna('True')

In [24]:
# The permit feature is missin a little over 5% of its values so they will be filled with a string
train['permit'] = train['permit'].fillna('Unsure')
test_data_copy['permit'] = test_data_copy['permit'].fillna('Unsure')

In [25]:

# We will drop train waterpoint_type because it is very close to waterpoint_type_group and redundency isn't good
train.drop(['waterpoint_type'], axis=1, inplace=True)
test_data_copy.drop(['waterpoint_type'], axis=1, inplace=True)

In [26]:
# Dropping the feature recorded_by because it has the same value which is not useful
train.drop(['recorded_by'], axis=1, inplace=True)
test_data_copy.drop(['recorded_by'], axis=1, inplace=True)

In [27]:
# Source and source type are very similar to the point where it seems some
# of the value groups have been grouped together so we will drop souce 

train.drop(['source'], axis=1, inplace=True)
test_data_copy.drop(['source'],axis=1, inplace=True)

In [28]:
train.drop(['quality_group'], axis=1, inplace=True)
test_data_copy.drop(['quality_group'], axis=1, inplace=True)

In [29]:
train.drop(['payment'], axis=1, inplace=True)
test_data_copy.drop(['payment'], axis=1, inplace=True)

In [30]:
# The extraction features are very similar and even seem to be a little mergered so we will drop two of them
train.drop(['extraction_type_group'], axis=1, inplace=True)
test_data_copy.drop(['extraction_type_group'], axis=1, inplace=True)

train.drop(['extraction_type'], axis=1, inplace=True)
test_data_copy.drop(['extraction_type'], axis=1, inplace=True)

In [ ]:
# We will drop management because management_group is the categorized version of the data
train.drop(['management'], axis=1, inplace=True)
test_data_copy.drop(['management'], axis=1, inplace=True)

In [31]:
def cat_construction(x):
     if x['construction_year'] >= int(1960) and x['construction_year'] < int(1970):
         return '60s'
     elif x['construction_year'] >= int(1970) and x['construction_year'] < int(1980):
         return '70s'
     elif x['construction_year'] >= int(1980) and x['construction_year'] < int(1990):
         return '80s'
     elif x['construction_year'] >= int(1990) and x['construction_year'] < int(2000):
         return '90s'
     elif x['construction_year'] >= int(2000) and x['construction_year'] < int(2010):
         return '00s'
     elif x['construction_year'] >= int(2010):
         return '10s'
     else:
         return 'unknown'
    
train['construction_year'] = train.apply(lambda x: cat_construction(x), axis=1)
test_data_copy['construction_year'] = test_data_copy.apply(lambda x: cat_construction(x), axis=1)

In [32]:
train['constuction_year'].value_counts

unknown    20709
00s        15330
90s         7678
80s         5578
10s         5161
70s         4406
60s          538
Name: construction_year, dtype: int64

In [39]:
train.drop(['subvillage'], axis=1, inplace=True)
test_data_copy.drop(['subvillage'], axis=1, inplace=True)

In [35]:
train.drop(['date_recorded'], axis=1, inplace=True)
test_data_copy.drop(['date_recorded'], axis=1, inplace=True)

In [40]:
train.columns

Index(['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'longitude',
       'latitude', 'wpt_name', 'num_private', 'basin', 'region', 'region_code',
       'district_code', 'lga', 'ward', 'population', 'public_meeting',
       'scheme_management', 'permit', 'construction_year',
       'extraction_type_class', 'management', 'management_group',
       'payment_type', 'water_quality', 'quantity', 'quantity_group',
       'source_type', 'source_class', 'waterpoint_type_group', 'status_group'],
      dtype='object')

In [37]:
train.head(1)

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,permit,construction_year,extraction_type_class,management,management_group,payment_type,water_quality,quantity,quantity_group,source_type,source_class,waterpoint_type_group,status_group
0,69572,6000.0,other,1390,other,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,vwc,False,90s,gravity,vwc,user-group,annually,soft,enough,enough,spring,groundwater,communal standpipe,functional


In [46]:
dis_col = [
           "funder",
           "installer",
           "wpt_name",
           "basin",
           "subvillage",
           "region_code",
           "district_code",
           "scheme_management",
           "permit",
           "construction_year",
           "management_group",
           "payment_type",
           "water_quality",
           "quantity",
           "quantity_group",
           "source_type",
           "source_class",
           "waterpoint_type_group"


]

In [52]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 31 columns):
id                       59400 non-null int64
amount_tsh               59400 non-null float64
funder                   59400 non-null object
gps_height               59400 non-null int64
installer                59400 non-null object
longitude                59400 non-null float64
latitude                 59400 non-null float64
wpt_name                 59400 non-null object
num_private              59400 non-null int64
basin                    59400 non-null object
region                   59400 non-null object
region_code              59400 non-null int64
district_code            59400 non-null int64
lga                      59400 non-null object
ward                     59400 non-null object
population               59400 non-null int64
public_meeting           59400 non-null object
scheme_management        59400 non-null object
permit                   59400 non-null object
c

In [72]:
if train['wpt_name'].str.contains(""'Iringa').any():
    print("Yes contains Iringa")
else:
      print('no')

SyntaxError: ignored

In [80]:
df1 = train['source_type'].str.contains('Iringa').sum()
print(df1)

0


In [43]:
ctgan = CTGANSynthesizer()


In [71]:
ctgan.fit(train, dis_col, epochs=1)

ValueError: ignored

In [ ]:
print('adfa')

adfa


SyntaxError: ignored

In [6]:
data = pd.read_csv('housing_dataset_cleaned.csv')

In [10]:
ctgan = CTGANSynthesizer()